In [1]:
import pymysql.cursors
import pandas as pd
import uuid
import time
import os
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
import pandas as pd

# import custom data connector
import data_connector

import string
import re
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import array as arr
from tqdm import tqdm


In [14]:
# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    # remove whitspace
    strOut = strOut.strip()
    # 
    strOut = re.sub('\s+',' ',strOut)
    
    return strOut

sql = '''
select 	id as ref_id, "description" as text_content
from google_result 
where monitoring_id = '115fd4ed-8608-4137-8170-e8d49238dece' -- 50
'''
# df = data_connector.execute_query_psql(sql % ('eb784a56-6fcb-4252-b0b8-21402b8c286f'))
df = data_connector.execute_query_psql(sql)
    
# Apply to data frame
df['text_content'] = df['text_content'].apply(text_preproc)
df.head()

from nltk.corpus import stopwords 

# Load stopwords
sw = stopwords.words('indonesian')

# apply tokenize
df['tokenized_tweet'] = df.apply(lambda row: nltk.word_tokenize(row['text_content']), axis=1)

# apply stopword removal
df['sw'] = df.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

df.head()

,ref_id,text_content,tokenized_tweet,sw
0,36457e02-c3de-41c6-ad50-722d2663f217,muhammadiyah or id jakarta – menanggapi terkai...,"[muhammadiyah, or, id, jakarta, –, menanggapi,...","{–, jakarta, keagamaan, mengelola, or, sekreta..."
1,f6fe390c-5ad8-4ed8-8d33-ab719f8e51c7,ketua pp muhammadiyah saad ibrahim memastikan ...,"[ketua, pp, muhammadiyah, saad, ibrahim, memas...","{usaha, mengelola, ibrahim, pemerintah, tamban..."
2,268934c6-eb53-40ff-837e-bd6905dffd49,pp muhammadiyah beberkan alasan pengalihan dan...,"[pp, muhammadiyah, beberkan, alasan, pengaliha...","{risiko, syariah, bank, berkaitan, indonesia, ..."
3,e3db7f85-5fd3-4d9b-b9d4-f16aae96b59b,bsi merespons keputusan pp muhammadiyah yang m...,"[bsi, merespons, keputusan, pp, muhammadiyah, ...","{mengalihkan, merespons, bank, keputusan, syar..."
4,b3f323f8-2526-43f5-a871-ddd18ca9199e,emiten perbankan syariah terbesar ri bank syar...,"[emiten, perbankan, syariah, terbesar, ri, ban...","{bris, syariah, terbesar, bank, emiten, terkai..."
